<a href="https://colab.research.google.com/github/CIS-522/course-content/blob/main/tutorials/W3_MLPs/W3_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIS-522 Week 3 Homework


**Instructor:** Konrad Kording

**Content Creators:** Arash Ash, Jordan Lei

In [ ]:
#@markdown What is your Pennkey and pod? (text, not numbers, e.g. bfranklin)
my_pennkey = 'value' #@param {type:"string"}
my_pod = 'euclidean-wombat' #@param ['Select', 'euclidean-wombat', 'sublime-newt', 'buoyant-unicorn', 'lackadaisical-manatee','indelible-stingray','superfluous-lyrebird','discreet-reindeer','quizzical-goldfish','ubiquitous-cheetah','nonchalant-crocodile','fashionable-lemur','spiffy-eagle','electric-emu','quotidian-lion','astute-jellyfish', 'quantum-herring']

# start timing
import time
try:t0;
except NameError: t0 = time.time()


---
# Setup

In [ ]:
# imports
import os
import csv
import json
import random
import pathlib

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, TensorDataset
from torchvision.utils import make_grid
from IPython.display import HTML, display

dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dev, torch.get_num_threads()

In [ ]:
# @title Install Kaggle API
%%capture
!pip install kaggle==1.5.10

In [ ]:
# @title Helper functions
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis(False)
    plt.show()

def progress(epoch, loss, epochs=100):
    return HTML("""
        <label for="file">Training loss: {loss}</label>
        <progress
            value='{epoch}'
            max='{epochs}',
            style='width: 100%'
        >
            {epoch}
        </progress>
    """.format(loss=loss, epoch=epoch, epochs=epochs))

## Part 1. Kaggle Competition
To use the Kaggle API, sign up for a Kaggle account at https://www.kaggle.com. Then go to the 'Account' tab of your user profile (`https://www.kaggle.com/<username>/account`) and select 'Create API Token'. This will trigger the download of `kaggle.json`, a file containing your API credentials. Place this file in Colab Files on the left menu.

Then join the competition and accept the rules at https://www.kaggle.com/c/permuted-animal-faces/rules.
Now we can handle the submisions here using the Kaggle API:

In [ ]:
# reegistering your API key
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c permuted-animal-faces
!unzip permuted-animal-faces.zip

Now it's your turn to train a competitive model using what you learned about MLPs and show it off to the rest of the world in the Kaggle leaderboard.

Here we provided you the same code as in the tutorial and the submission code, which you can run and get a baseline result. But we encourage you to get creative and combine your previous ML know-how since nothing is off the table in this competition.

In [ ]:
class Net(nn.Module):
    def __init__(self, actv, num_inputs, hidden_units, num_outputs):
        super(Net, self).__init__()

        exec('self.actv = nn.%s'%actv)   # [TO-DO]

        self.layers = nn.ModuleList()
        for i in range(len(hidden_units)):
          next_num_inputs = hidden_units[i] 
          self.layers += [nn.Linear(num_inputs, next_num_inputs)]   # [TO-DO]
          num_inputs = next_num_inputs

        self.out = nn.Linear(num_inputs, num_outputs)

    def forward(self, x):
        # flattening
        x = x.view(x.shape[0], -1)   # [TO-DO]

        for layer in self.layers:
          x = self.actv(layer(x))  # [TO-DO]
        x = self.out(x) # [TO-DO]
        return x

In [ ]:
df = pd.read_csv('train.csv')
df.head()

In [ ]:
data = np.array(df)
X_train = torch.tensor(data[:, 1:]).float()/255
y_train = torch.tensor(data[:, 0]).long()

In [ ]:
batch_size = 128
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, drop_last=True,
                        shuffle=True, num_workers=10)

In [ ]:
def train_test_classification(net, criterion, optimizer,
                              train_loader,
                              num_epochs=1, verbose=True,
                              training_plot=True):
  if verbose:
    progress_bar = display(progress(0, 0, num_epochs), display_id=True)

  net.train()   # [TO-DO]
  training_losses = []
  for epoch in range(num_epochs):  # loop over the dataset multiple times
      running_loss = 0.0
      for i, data in enumerate(train_loader, 0):
          # get the inputs; data is a list of [inputs, labels]
          inputs, labels = data
          inputs = inputs.to(dev).float()    # [TO-DO]
          labels = labels.to(dev).long()   # [TO-DO]

          # zero the parameter gradients
          optimizer.zero_grad()   # [TO-DO]

          # forward + backward + optimize
          outputs = net(inputs)   # [TO-DO]

          loss = criterion(outputs, labels)   # [TO-DO]
          loss.backward()   # [TO-DO]
          optimizer.step()   # [TO-DO]

          # print statistics
          if verbose:
            training_losses += [loss.item()]
            running_loss += loss.item()
            if i % 10 == 9:    # update every 10 mini-batches
                progress_bar.update(progress(epoch+1, running_loss / 10, num_epochs))
                running_loss = 0.0

  net.eval()   # [TO-DO]
  def test(data_loader):
    correct = 0
    total = 0
    for data in data_loader:
        inputs, labels = data
        inputs = inputs.to(dev).float()
        labels = labels.to(dev).long()

        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)   # [TO-DO]
        total += labels.size(0)   # [TO-DO]
        correct += (predicted == labels).sum().item()   # [TO-DO]

    acc = 100 * correct / total    # [TO-DO]
    return total, acc

  train_total, train_acc = test(train_loader)

  if verbose:
    print('Accuracy on the %d training samples: %0.2f %%' % (train_total, train_acc))

  if training_plot:
    plt.plot(training_losses)
    plt.xlabel('Batch')
    plt.ylabel('Training loss')
    plt.show()
  
  return train_acc

In [ ]:
net = Net('ReLU()', 3*32*32, [128, 32], 3).to(dev) 
criterion = nn.MultiMarginLoss(margin=1.0)
optimizer = optim.Adam(net.parameters(), lr=3e-4)
train_acc = train_test_classification(net, criterion, optimizer,
                                      train_loader,
                                      num_epochs=50)

In [ ]:
df = pd.read_csv('test.csv')
data = np.array(df)
X_test = torch.tensor(data).float()/255

net.eval()
y_pred = net(X_test)
labels_pred = torch.argmax(y_pred, dim=1)

In [ ]:
header = ['ImageId', 'Label']
with open('submission.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    
    for i in range(len(labels_pred)):
      writer.writerow([i+1] + [labels_pred[i].item()])

In [ ]:
!kaggle competitions submit permuted-animal-faces -f submission.csv -m "Result with same model as in the tutorial"

Note that you can submit maximum 20 times a day. Participants will need to wait until the next UTC day after submitting the maximum number of daily submissions.

And finally you can hand select 2 submissions among your previous ones to be used for the final ranking.

## Part 2.